In [ ]:
%reload_ext dotenv
%dotenv

In [ ]:
from PIL import Image
from google import genai
from google.genai import types

In [ ]:
model_im = Image.open("../images/humans/kelsey.jpeg")
model_im.thumbnail((400, 400))
model_im

In [ ]:
from io import BytesIO

client = genai.Client()

prompt = 'style the person as a sims 3 character, no text/UI/diamond above the head, video game style (PS3), not photorealistic, not cell-shaded, preserve face details, plain light-gray background, no objects other than the person, full body (head to toe), soft lighting, relaxed pose with arms by side, wearing white 9" inseam shorts and white regular fit t-shirt, white socks, no shoes/accessories, 10 degree angle facing slightly right (from camera perspective), looking straight ahead (not in camera), 512px resolution'
response = client.models.generate_content(
    model="gemini-3.1-flash-image-preview",
    contents=[model_im, prompt],
    config=types.GenerateContentConfig(
        image_config=types.ImageConfig(
            aspect_ratio="3:4",
            image_size="512px",
        ),
    ),
)

assert response.parts is not None

for part in response.parts:
    if part.text is not None:
        print(part.text)
    elif part.inline_data is not None:
        genai_image = part.as_image()
        assert genai_image is not None
        assert genai_image.image_bytes is not None
        image = Image.open(BytesIO(genai_image.image_bytes))

usage = response.usage_metadata
assert usage is not None
print(
    f"\nToken usage: {usage.prompt_token_count} input, {usage.candidates_token_count} output, {usage.total_token_count} total"
)
print(f"Image size: {image.size} px")
preview = image.copy()
preview.thumbnail((600, 600))
preview